### קוד מבוא

#### ספריות

In [27]:
import os
import sys
import pathlib
import pandas as pd

In [28]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

#### העלאת משתנים להרצת הקוד

In [29]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

future_folder = os.path.dirname(software_data_folder_location)

create_forecast_basic_software_location = os.path.dirname(future_folder)

sys.path.append(future_folder)

future_df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(future_folder))

TAZ_V4_date = future_df_inputs_outputs['location'][1]


### פונקציות

### פונקציות גלובליות

In [30]:
from global_functions import make_point, up_load_shp, up_load_df, find_files_with_pattern

### ביצוע

#### העלאת שכבות

In [31]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_software_location, TAZ_V4_date))

In [32]:
taz=taz.query('main_secto=="Arab" | main_secto=="arabs_behined_seperation_wall" ').set_index('Taz_num')

taz_point=make_point(taz)

#### יצירת אוכלוסיה לפי קיבולת

תוספת קיבולת אפשרית

In [33]:
path=r'{}\background_files\kibolet_with_behind_wall_230709.shp'.format(software_data_folder_location)

kibolet_with_behind_wall=up_load_shp(path).set_index('Taz_num')

יח"ד דיור קיימות

In [34]:
matching_files=find_files_with_pattern(r'{}'.format(forecast_version_folder_location), '2020_jtmt_forcast_full')

file = pathlib.Path(matching_files[0])
file_name = file.name[:-5]

taz_kayim=up_load_df(forecast_version_folder_location,file_name).set_index('Taz_num')

ג"מ

In [35]:
path=r'{}\background_files\hh_size_2050.shp'.format(software_data_folder_location)

hh_size_2050=up_load_shp(path)

In [36]:
hh_size_2050=taz_point.sjoin(hh_size_2050,how='left')[['2050_hh_si']]

taz['hh_size']=hh_size_2050['2050_hh_si']

הכנה ללופ : מצב קיים הופך למצב נוכחי

In [37]:
taz['current_pop']=taz_kayim['pop']

taz['pop_2020']=taz_kayim['pop']

taz['current_hh']=taz_kayim['aprt_20']

taz['hh_2020']=taz_kayim['aprt_20']

להוציא את האזורים שמחוץ לירושלים

In [38]:
taz_out_of_jeru=taz.loc[taz.index.isin([7651.0, 7652.0])]

In [39]:
taz=taz.loc[~taz.index.isin([7651.0, 7652.0])]

עכשיו חישוב של כמות אוכלוסיה כל חומש

In [40]:
taz['pop_kibolet']=taz['hh_size']*kibolet_with_behind_wall['kibolet']

הכנה ללופ : הלאה של אוכלסייה פלסטינאית

In [41]:
folder_path=r'{}\background_files'.format(software_data_folder_location)
file_name='taz_demo_pls_2020_and_pre_growth_till_2050'

taz_demo_pls=up_load_df(folder_path,file_name)

taz_demo_pls['current_pop']=taz_demo_pls['pop_2020']

#### לופ לייצר אנשים כל חומש

In [42]:
col_age=['Taz_num','pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

חישוב מחוץ לירושלים

In [43]:
taz_out_of_jeru_point=make_point(taz_out_of_jeru)

In [44]:
year=['2025','2030','2035','2040','2045','2050']

for y in year:
    # לחבר למצב הנוכחי את הגידול והתפלגות שהוא צריך לקבל
    path=r'{}\background_files\pre_demo_growth_dislov_{}.shp'.format(software_data_folder_location, y)

    demo=up_load_shp(path)  

    demand=taz_out_of_jeru_point.sjoin(demo,how='left')

    taz_age=demand.reset_index()[col_age]
    
    taz_out_of_jeru_point['add_current_pop']=taz_out_of_jeru_point['current_pop']*demand['precent_ad']-taz_out_of_jeru_point['current_pop']
    
    taz_out_of_jeru_point['current_pop']=taz_out_of_jeru_point['current_pop']+taz_out_of_jeru_point['add_current_pop']
    
    taz_out_of_jeru_point['current_hh']=taz_out_of_jeru_point['current_hh']+taz_out_of_jeru_point['add_current_pop']/taz_out_of_jeru_point['hh_size']
    
    taz_out_of_jeru_point['hh_{}'.format(y)]=taz_out_of_jeru_point['current_hh']
    
    taz_out_of_jeru_point['pop_{}'.format(y)]=taz_out_of_jeru_point['current_pop']
    
    col=['Taz_num','hh_{}'.format(y),'pop_{}'.format(y)]

    locals()['taz_out_jeru_{}'.format(y)]=taz_out_of_jeru_point.reset_index()[col].merge(taz_age,how='left',on='Taz_num')   

חישוב לירושלים ואז הוספה של המרכיבים

In [45]:
#מייצר קטלוג לאזורים ללא התפלגות מדמוגרף
age_dis=up_load_df(r'{}\background_files'.format(software_data_folder_location),'age_des_types')

age_col_catalog=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

age_dis=age_dis.query('sector=="Arab" & age_cycle==2')[age_col_catalog]

In [46]:
taz['precent_ad']=0

In [47]:
x=2020

In [48]:
year=['2025','2030','2035','2040','2045','2050']

In [49]:
for y in year:
    # לחבר למצב הנוכחי את הגידול והתפלגות שהוא צריך לקבל
    path=r'{}\background_files\pre_demo_growth_dislov_{}.shp'.format(software_data_folder_location, y)

    demo=up_load_shp(path)

    taz_point=make_point(taz.reset_index()[['Taz_num','geometry']])

    demand=taz_point.sjoin(demo,how='left')
    taz_age=demand.reset_index()[col_age]
    
    demand=demand[['Taz_num','precent_ad']].set_index('Taz_num')

    taz['precent_ad']=demand['precent_ad']
    taz['precent_ad']=taz['precent_ad'].fillna(0)
    taz['precent_ad_{}'.format(y)]=taz['precent_ad']

    taz.loc[taz['precent_ad']>0,'pop_demand']=taz['current_pop']*taz['precent_ad']-taz['current_pop']
    taz.loc[taz['precent_ad']<=0,'pop_demand']=0
    taz['pop_demand_{}'.format(y)]=taz['pop_demand']
    taz['pop_kibolet_{}'.format(y)]=taz['pop_kibolet']

    # יצירת התפלגות,גידול וג"מ הממוצע של כל אחד מהערכים האלו בחומש
    # אזורים עם אוכלוסייה וללא התפלגות,גידול וג"מ יקבלו את הערך הממוצע הנל
    # מצב נוכחי גדל לפי אחוז
    # השוואה בין אוכלוסית ביקוש לקיבולת
    print("------------------------------------------")
    print(y)
    print("total_kibolet: ",taz['pop_kibolet'].sum().item())
    print("total_demand: ",taz['pop_demand'].sum().item())
    
    taz['pop_delta']=taz['pop_kibolet']-taz['pop_demand']
    taz['pop_delta_{}'.format(y)]=taz['pop_delta']

    # יש מקום לביקוש - לקחת את הביקוש
    taz.loc[taz['pop_delta']>0,'current_pop']=taz['pop_demand']+taz['current_pop']
    taz.loc[taz['pop_delta']>0,'pop_kibolet']=taz['pop_kibolet']-taz['pop_demand']
    taz.loc[taz['pop_delta']>0,'pop_{}_explanation'.format(y)]='pop_demand'


    # אין מקום לביקוש - לקחת את הקיבולת
    taz.loc[(taz['pop_delta']<=0)&(taz['pop_kibolet']>0),'current_pop']=taz['pop_kibolet']+taz['current_pop']
    taz.loc[(taz['pop_delta']<=0)&(taz['pop_kibolet']>0),'pop_kibolet']=0
    taz.loc[(taz['pop_delta']<=0)&(taz['pop_kibolet']>0),'pop_{}_explanation'.format(y)]='full_pop_kibolet'

    # לקבץ את כלל האוכלוסיה ללא בית לסל אחד
    pop_not_fit_in_taz=taz.loc[taz['pop_delta']<=0]['pop_delta'].sum().item()
    
    pop_not_fit_in_taz=abs(pop_not_fit_in_taz)
    
    print("pop_not_fit_in_taz: ",pop_not_fit_in_taz)

    # כימות כמה עוד מקום יש בקיבולת של ירושלים
    pop_kibolet_left=taz['pop_kibolet'].sum().item()
    
    print("pop_kibolet_left: ",pop_kibolet_left)

    if pop_not_fit_in_taz>pop_kibolet_left:
        print("pop_not_fit_in_taz>pop_kibolet_left")

        taz['current_pop']=taz['pop_kibolet']+taz['current_pop']
        taz['pop_kibolet']=0
        taz['pop_{}_explanation'.format(y)]=taz['pop_{}_explanation'.format(y)]+' +add_pop_to_fill_kibolet'
                
        pop_not_fit_in_jeru=pop_not_fit_in_taz-pop_kibolet_left
        
        print("pop_not_fit_in_jeru: ",pop_not_fit_in_jeru)
        
        taz_demo_pls['current_pop_pre']=taz_demo_pls['current_pop']/taz_demo_pls['current_pop'].sum().item()
        taz_demo_pls['add_pop_{}_from_jeru'.format(y)]=taz_demo_pls['current_pop_pre']*pop_not_fit_in_jeru
        taz_demo_pls['current_pop']=taz_demo_pls['current_pop']*taz_demo_pls['precent_add_pop_{}'.format(y)]
        taz_demo_pls['current_pop']=taz_demo_pls['current_pop']+taz_demo_pls['add_pop_{}_from_jeru'.format(y)]
        taz_demo_pls['pop_{}'.format(y)]=taz_demo_pls['current_pop']
    else:
        print("pop_not_fit_in_taz<=pop_kibolet_left")

        taz['add_pop_kibolet_from_pre']=taz['pop_kibolet']*(pop_not_fit_in_taz/pop_kibolet_left)
        taz['add_pop_kibolet_from_pre_{}'.format(y)]=taz['add_pop_kibolet_from_pre']
        taz['current_pop']=taz['add_pop_kibolet_from_pre']+taz['current_pop']
        taz['pop_kibolet']=taz['pop_kibolet']-taz['add_pop_kibolet_from_pre']
        taz.loc[taz['add_pop_kibolet_from_pre']>0,'pop_{}_explanation'.format(y)]=taz['pop_{}_explanation'.format(y)]+' +add_pop_by_pre_of_kibolet_pre'
        
        taz_demo_pls['current_pop']=taz_demo_pls['current_pop']*taz_demo_pls['precent_add_pop_{}'.format(y)]
        taz_demo_pls['pop_{}'.format(y)]=taz_demo_pls['current_pop']        
    
    taz['pop_{}'.format(y)]=taz['current_pop']
    taz['add_pop_{}_{}'.format(y,str(x))]=taz['current_pop']-taz['pop_{}'.format(str(x))]
    taz.loc[(taz['add_pop_{}_{}'.format(y,str(x))]>0)&(taz['hh_size']<=0),'current_hh']=taz['current_hh']+taz['add_pop_{}_{}'.format(y,str(x))]/4.5
    
    taz.loc[(taz['add_pop_{}_{}'.format(y,str(x))]>0)&(taz['hh_size']>0),'current_hh']=taz['current_hh']+taz['add_pop_{}_{}'.format(y,str(x))]/4.5
    
    taz['hh_{}'.format(y)]=taz['current_hh']
 
    x=x+5
    col_current=['Taz_num','hh_{}'.format(y),'pop_{}'.format(y)]
    col=['pop_0',
     'pop_10',
     'pop_15',
     'pop_20',
     'pop_25',
     'pop_30',
     'pop_35',
     'pop_40',
     'pop_45',
     'pop_5',
     'pop_50',
     'pop_55',
     'pop_60',
     'pop_65',
     'pop_70',
     'pop_75up']
    
    df=taz.reset_index()[col_current].merge(taz_age,how='left',on='Taz_num')
    
    df=pd.concat([df,locals()['taz_out_jeru_{}'.format(y)]],axis=0)

    df1=df.loc[((df[col].sum(axis=1)==0)&(df['pop_{}'.format(y)]>0))]

    df1=df1[col_current]

    df1['join']=1

    age_dis['join']=1

    df1=df1.merge(age_dis,on='join',how='left').drop(columns='join')

    df=df.loc[~((df[col].sum(axis=1)==0)&(df['pop_{}'.format(y)]>0))]

    df=pd.concat([df,df1],axis=0)

    df=df.round(2)
    
    df[col]=df[col].div(df[col].sum(axis=1), axis=0)
    
    df[col] = df[col].apply(lambda x: x * df['pop_{}'.format(y)])
    
    df['pop_{}'.format(y)]=df[col].sum(axis=1)
           
    df=pd.concat([df,taz_demo_pls[['Taz_num','pop_{}'.format(y)]]],axis=0)  
    
    col=['Taz_num','hh_{}'.format(y),'pop_{}'.format(y),'pop_0',
         'pop_10',
         'pop_15',
         'pop_20',
         'pop_25',
         'pop_30',
         'pop_35',
         'pop_40',
         'pop_45',
         'pop_5',
         'pop_50',
         'pop_55',
         'pop_60',
         'pop_65',
         'pop_70',
         'pop_75up']
        
    df[col].to_excel(r'{}\Intermediates\taz_Arab_Palestinian_{}.xlsx'.format(software_data_folder_location, y),index=False)

------------------------------------------
2025
total_kibolet:  235869.44800000003
total_demand:  47430.294430891474
pop_not_fit_in_taz:  343.6068327890348
pop_kibolet_left:  188782.76040189757
pop_not_fit_in_taz<=pop_kibolet_left
------------------------------------------
2030
total_kibolet:  188439.15356910854
total_demand:  51025.89936713023
pop_not_fit_in_taz:  2707.642208727688
pop_kibolet_left:  140120.896410706
pop_not_fit_in_taz<=pop_kibolet_left


C:\Users\dpere\AppData\Local\Temp\ipykernel_35720\198304994.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'pop_demand' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz.loc[taz['pop_delta']>0,'pop_{}_explanation'.format(y)]='pop_demand'
C:\Users\dpere\AppData\Local\Temp\ipykernel_35720\198304994.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'pop_demand' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz.loc[taz['pop_delta']>0,'pop_{}_explanation'.format(y)]='pop_demand'


------------------------------------------
2035
total_kibolet:  137413.25420197833
total_demand:  51121.438766827094
pop_not_fit_in_taz:  13332.138296832742
pop_kibolet_left:  99623.95373198394
pop_not_fit_in_taz<=pop_kibolet_left
------------------------------------------
2040
total_kibolet:  86291.8154351512
total_demand:  49927.5882063552
pop_not_fit_in_taz:  25023.76916806403
pop_kibolet_left:  61387.996396860035
pop_not_fit_in_taz<=pop_kibolet_left


C:\Users\dpere\AppData\Local\Temp\ipykernel_35720\198304994.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'pop_demand' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz.loc[taz['pop_delta']>0,'pop_{}_explanation'.format(y)]='pop_demand'
C:\Users\dpere\AppData\Local\Temp\ipykernel_35720\198304994.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'pop_demand' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz.loc[taz['pop_delta']>0,'pop_{}_explanation'.format(y)]='pop_demand'


------------------------------------------
2045
total_kibolet:  36364.22722879601
total_demand:  48230.6222676434
pop_not_fit_in_taz:  34904.56295391077
pop_kibolet_left:  23038.16791506338
pop_not_fit_in_taz>pop_kibolet_left
pop_not_fit_in_jeru:  11866.39503884739
------------------------------------------
2050
total_kibolet:  0
total_demand:  46798.07357171309
pop_not_fit_in_taz:  46798.07357171309
pop_kibolet_left:  0
pop_not_fit_in_taz>pop_kibolet_left
pop_not_fit_in_jeru:  46798.07357171309


C:\Users\dpere\AppData\Local\Temp\ipykernel_35720\198304994.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'pop_demand' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz.loc[taz['pop_delta']>0,'pop_{}_explanation'.format(y)]='pop_demand'
C:\Users\dpere\AppData\Local\Temp\ipykernel_35720\198304994.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'pop_demand' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz.loc[taz['pop_delta']>0,'pop_{}_explanation'.format(y)]='pop_demand'
